# Part I. Pre-Processing the Files

#### Import Python packages 

In [39]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [40]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [41]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [42]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Creating tables, insert data and execute test queries

The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the **event_datafile_new.csv** after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Creating a Cluster

In [43]:
from cassandra.cluster import Cluster
cluster = Cluster()
session = cluster.connect()

## Create Keyspace

In [44]:
session.execute("CREATE KEYSPACE IF NOT EXISTS sparkifydb \
                 WITH REPLICATION = { \
                                     'class' : 'SimpleStrategy', \
                                     'replication_factor' : 1 \
                                     }")

## Set Keyspace

In [45]:
session.set_keyspace("sparkifydb")

## Create queries

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

- When I designed the table for this query I chose the *session_id* as PARTITION KEY and *item_in_session* as CLUSTERING COLUMN.
- This two information is enough to uniquely identify a row.
- Using *session_id* as PARTITION KEY is good, because rows related to a given session will be stored on a single data node (or more if the session is very long).


In [47]:
query = "CREATE TABLE IF NOT EXISTS session_table (artist text, \
                                                   song_title text, \
                                                   song_length float, \
                                                   session_id int, \
                                                   item_in_session int, \
                                                   PRIMARY KEY (session_id, item_in_session))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [48]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_table (artist, song_title, song_length, session_id, item_in_session)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

#### Execute test query

In [50]:
query = "SELECT artist, song_title, song_length FROM session_table WHERE session_id = 338 and item_in_session = 4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print((row.artist, row.song_title, row.song_length))

('Faithless', 'Music Matters (Mark Knight Dub)', 495.30731201171875)


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

- When I designed the table for this query I chose the *session_id* as PARTITION KEY and *item_in_session* as CLUSTERING COLUMN.
- This three information is enough to uniquely identify a row.
- Using *session_id* as PARTITION KEY is good, because rows related to a given session will be stored on a single data node (or more if the session is very long).

In [51]:
query = "CREATE TABLE IF NOT EXISTS session_playlist_table (artist_name text, \
                                                song_title text, \
                                                user_first_name text, \
                                                user_last_name text, \
                                                user_id int, \
                                                session_id int, \
                                                item_in_session int, \
                                                PRIMARY KEY ((user_id, session_id), item_in_session)\
                                                ) WITH CLUSTERING ORDER BY (item_in_session ASC)"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [52]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_playlist_table (artist_name, song_title, user_first_name, user_last_name, user_id, session_id, item_in_session)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], line[1], line[4], int(line[10]), int(line[8]), int(line[3])))                  

#### Execute test query

In [54]:
query = "SELECT artist_name, song_title, user_first_name, user_last_name FROM session_playlist_table WHERE user_id = 10 AND session_id = 182"

try:
    rows = session.execute(query)
except Exception as e:
    pritn(e)
    
for row in rows:
    print((row.artist_name, row.song_title, row.user_first_name, row.user_last_name))

('Down To The Bone', "Keep On Keepin' On", 'Sylvie', 'Cruz')
('Three Drives', 'Greece 2000', 'Sylvie', 'Cruz')
('Sebastien Tellier', 'Kilometer', 'Sylvie', 'Cruz')
('Lonnie Gordon', 'Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', 'Sylvie', 'Cruz')


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

- When I designed the table for this query I chose the *song_title* as PARTITION KEY and *user_id* as CLUSTERING COLUMN.
- This two information is enough to uniquely identify a row.
- Using *song_title* as PARTITION KEY is good, because rows related to a given song will be stored on a single data node (or more if the song is very popular).
- Another benefit is that in this way we can use *song_title* column first in a WHERE clause.

In [55]:
query = "CREATE TABLE  IF NOT EXISTS song_table (user_first_name text, \
                                                 user_last_name text, \
                                                 song_title text, \
                                                 user_id int, \
                                                 PRIMARY KEY (song_title, user_id))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [56]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_table (user_first_name, user_last_name, song_title, user_id)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[1], line[4], line[9], int(line[10])))

In [58]:
query = "SELECT user_first_name, user_last_name FROM song_table WHERE song_title = 'All Hands Against His Own'"

try:
    rows = session.execute(query)
    error = True
except Exception as e:
    print(e)
    error = False

if error:
    for row in rows:
        print(row.user_first_name, row.user_last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [59]:
query = "DROP TABLE IF EXISTS session_table"

try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS user_table"

try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS song_table"

try:
    result = session.execute(query)
except Exception as e:
    print(e)

In [60]:
query = "DROP KEYSPACE IF EXISTS sparkifydb"

try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [61]:
session.shutdown()
cluster.shutdown()